In [14]:
# Import dependencies
import pandas as pd
import requests
import numpy as np

In [15]:
# Read data from csv
filepath = 'Real-Time_Traffic_Incident_Reports.csv'
df = pd.read_csv(filepath)

In [16]:
# View
df

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,06/13/2018 06:35:59 AM +0000,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,ARCHIVED,06/13/2018 09:00:03 AM +0000
1,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,06/13/2018 10:15:36 AM +0000,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,ARCHIVED,06/13/2018 11:20:03 AM +0000
2,2A7A84FC69D56A2C379C274D2F0831442D3E7B8E_15871...,04/17/2020 09:25:03 PM +0000,Crash Urgent,"(30.329455,-97.638105)",30.329455,-97.638105,E US 290 HWY SVRD EB & EASTERN HEIGHTS BLVD,ARCHIVED,04/17/2020 09:55:03 PM +0000
3,89D3B75BB5932F10D25B5852A9902DA26C3DF11A_15871...,04/17/2020 09:40:52 PM +0000,Traffic Hazard,"(30.202806,-97.760701)",30.202806,-97.760701,5300-blk S Ih 35 Sb,ARCHIVED,04/17/2020 09:55:03 PM +0000
4,E7F9EFF0E81057A5C8FE41CABE4D9C6EEA82DAA0_15871...,04/17/2020 09:00:55 PM +0000,Crash Urgent,"(30.184265,-97.687339)",30.184265,-97.687339,4500-4720 S Us 183 Hwy Sb,ARCHIVED,04/17/2020 11:30:04 PM +0000
...,...,...,...,...,...,...,...,...,...
354860,08E13D6D910F8F4B334B0C59104105E88B52CDB5_17024...,12/13/2023 07:19:24 PM +0000,Crash Urgent,"(30.157594,-97.789847)",30.157594,-97.789847,9811 S Ih 35 Svrd Nb,ACTIVE,12/13/2023 07:25:09 PM +0000
354861,08035191DD04AF9D68F7F8A779CAE1181F1937A1_17024...,12/13/2023 07:22:45 PM +0000,Traffic Hazard,"(30.264879,-97.757016)",30.264879,-97.757016,100-132 S Lamar Blvd,ACTIVE,12/13/2023 07:25:09 PM +0000
354862,91108D18FCC80F97E015B3E24F56361F312CFF22_17024...,12/13/2023 06:25:57 PM +0000,Traffic Hazard,"(30.337567,-97.71885)",30.337567,-97.718850,Airport Blvd / N Lamar Blvd,ARCHIVED,12/13/2023 07:30:12 PM +0000
354863,7C83F057E0C3BCAC5D8322031E3E959FD0E96716_17024...,12/13/2023 07:32:26 PM +0000,Traffic Hazard,"(30.19704,-97.787889)",30.197040,-97.787889,S 1st St / W William Cannon Dr,ACTIVE,12/13/2023 07:35:09 PM +0000


In [5]:
# Drop rows wth NaN values
df.dropna(axis=0, inplace=True)

In [6]:
# Convert dates to datetime format
df['Published Date'] = pd.to_datetime(df['Published Date'].str[:22], format='%m/%d/%Y %I:%M:%S %p')
df['Status Date'] = pd.to_datetime(df['Status Date'].str[:22], format='%m/%d/%Y %I:%M:%S %p')

In [7]:
pd.DataFrame(df['Issue Reported'].value_counts())

,count
Issue Reported,
Traffic Hazard,106580
Crash Urgent,84880
Crash Service,49524
COLLISION,34610
TRFC HAZD/ DEBRIS,25930
LOOSE LIVESTOCK,9791
COLLISION WITH INJURY,8891
zSTALLED VEHICLE,8658
Stalled Vehicle,6728


In [8]:
df['Latitude'].sort_values(ascending=True)

321676    3.000303e+01
37413     3.000388e+01
53992     3.000441e+01
204546    3.000462e+01
252461    3.000480e+01
              ...     
315188    2.999550e+07
354588    2.999582e+07
20517     2.999586e+07
258537    2.999586e+07
229050    2.999603e+07
Name: Latitude, Length: 352366, dtype: float64

In [9]:
# Remove Latitude data out of range
df = df.loc[df['Latitude'].abs() < 90]

In [10]:
# Filter out dates
df_sample = df.loc[df['Published Date'] > pd.Timestamp('10/31/2023')]
df_sample

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
292111,04085CEF4A2F22B0D7DE2D6919476571400985C3_16993...,2023-11-07 12:36:04,Stalled Vehicle,"(30.250913,-97.80653)",30.250913,-97.806530,3300 S CAPITAL OF TEXAS HWY NB,ARCHIVED,2023-11-07 13:25:09
292112,C465CA6FDF91527E8237C635BDF908249BA0A095_16996...,2023-11-10 10:54:42,Crash Service,"(30.334875,-97.690602)",30.334875,-97.690602,1300 E Anderson Ln Svrd Wb,ARCHIVED,2023-11-10 13:15:13
292381,97B0FD2F9B2C4E92174719458B283189C944E15B_16993...,2023-11-07 12:07:50,Crash Urgent,"(30.409475,-97.648629)",30.409475,-97.648629,E Howard Ln / Harris Ridge Blvd,ARCHIVED,2023-11-07 13:10:08
292382,C94C3FF3186FA0560180BA638775323FC4895576_16993...,2023-11-07 12:55:58,Stalled Vehicle,"(30.382712,-97.6979)",30.382712,-97.697900,Kramer Ln / Parkfield Dr,ARCHIVED,2023-11-07 13:10:08
292658,24234AC444DF5B42D02CF9947F4CB143EECC801C_16993...,2023-11-07 12:27:15,Crash Urgent,"(30.27117,-97.735448)",30.271170,-97.735448,Red River St / E 11th St,ARCHIVED,2023-11-07 13:25:09
...,...,...,...,...,...,...,...,...,...
354860,08E13D6D910F8F4B334B0C59104105E88B52CDB5_17024...,2023-12-13 19:19:24,Crash Urgent,"(30.157594,-97.789847)",30.157594,-97.789847,9811 S Ih 35 Svrd Nb,ACTIVE,2023-12-13 19:25:09
354861,08035191DD04AF9D68F7F8A779CAE1181F1937A1_17024...,2023-12-13 19:22:45,Traffic Hazard,"(30.264879,-97.757016)",30.264879,-97.757016,100-132 S Lamar Blvd,ACTIVE,2023-12-13 19:25:09
354862,91108D18FCC80F97E015B3E24F56361F312CFF22_17024...,2023-12-13 18:25:57,Traffic Hazard,"(30.337567,-97.71885)",30.337567,-97.718850,Airport Blvd / N Lamar Blvd,ARCHIVED,2023-12-13 19:30:12
354863,7C83F057E0C3BCAC5D8322031E3E959FD0E96716_17024...,2023-12-13 19:32:26,Traffic Hazard,"(30.19704,-97.787889)",30.197040,-97.787889,S 1st St / W William Cannon Dr,ACTIVE,2023-12-13 19:35:09


In [11]:
pd.DataFrame(df_sample['Issue Reported'].value_counts())

,count
Issue Reported,
Traffic Hazard,1649
Crash Urgent,1634
COLLISION,822
Crash Service,792
TRFC HAZD/ DEBRIS,500
LOOSE LIVESTOCK,233
Stalled Vehicle,213
COLLISION WITH INJURY,208
COLLISN/ LVNG SCN,165


In [12]:
df_sample.to_csv('austin_traffic_incident_reports_nov2023_sample.csv')